# This is the Xface Telegram Bot
###Made with 💖 using Python

# Install Dependencies

In [ ]:
!pip install wget python-telegram-bot==13.15 -q

#Main Bot Code 🤖

##Enter the Telegram Bot Token 💖

In [ ]:
# Set up your bot token
BOT_TOKEN = 'ENTER YOUR BOT TOKEN WITH 💖'

##Run the Code 🚀

In [ ]:
import cv2
import wget
import os
from telegram import Update, InputFile
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters, CallbackContext
from deepface import DeepFace

# Set up your bot token
BOT_TOKEN = '7018789296:AAGig2oPl6tTxEf09e-E5xrdS3-KU4nb3Y8'


#it starts the bot
def start(update: Update, context: CallbackContext) -> None:
    update.message.reply_text('Send me an image as a document or a URL to analyze the emotion.')

#this remotely downloads the image provided from the url
def download_image(url, output_dir="downloaded_images"):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    image_filename = wget.download(url, out=output_dir)
    return image_filename

#it uses the deepface library
def detect_emotion(image_path):
    try:
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Could not load image from path: {image_path}")
        result = DeepFace.analyze(image, actions=['emotion'])
        return result[0]['dominant_emotion']
    except Exception as e:
        return f"Error: {e}"

#document handler
def handle_document(update: Update, context: CallbackContext) -> None:
    document = update.message.document
    file = context.bot.get_file(document.file_id)
    file_path = f"downloaded_images/{document.file_name}"
    file.download(file_path)

    emotion = detect_emotion(file_path)
    update.message.reply_text(f"Detected Emotion: {emotion}")

#url handler
def handle_url(update: Update, context: CallbackContext) -> None:
    url = update.message.text
    file_path = download_image(url)

    emotion = detect_emotion(file_path)
    update.message.reply_text(f"Detected Emotion: {emotion}")

#main function
def main() -> None:
    updater = Updater(BOT_TOKEN)

    dispatcher = updater.dispatcher
    dispatcher.add_handler(CommandHandler("start", start))
    dispatcher.add_handler(MessageHandler(Filters.document.image, handle_document))
    dispatcher.add_handler(MessageHandler(Filters.text & ~Filters.command, handle_url))

    updater.start_polling()
    print("Bot is running...")
    updater.idle()

if __name__ == '__main__':
    main()